In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ1: Which technologies have been investigated in the last decade?

In [3]:
data = pd.read_csv('./data/export1.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Document Title               1946 non-null   object 
 1   Authors                      1804 non-null   object 
 2   Author Affiliations          1784 non-null   object 
 3   Publication Title            1946 non-null   object 
 4   Date Added To Xplore         1946 non-null   object 
 5   Publication Year             1946 non-null   int64  
 6   Volume                       652 non-null    float64
 7   Issue                        652 non-null    float64
 8   Start Page                   1946 non-null   object 
 9   End Page                     1944 non-null   object 
 10  Abstract                     1933 non-null   object 
 11  ISSN                         652 non-null    object 
 12  ISBNs                        1294 non-null   object 
 13  DOI               

In [4]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2))
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    df['PublishingYear'] = data['Year']
    returndf = df.groupby('PublishingYear').sum().transpose()
    return returndf

In [5]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [6]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

In [7]:
#Untersuchung Abstract

#abstract = CreateTermDocMatrix('Abstract')

#b = abstract.loc[abstract[2017] > 5] 

#b.plot.bar()
#print(b)

In [8]:
#Untersuchung Title
#title = CreateTermDocMatrix('Title')

#print(title)
#title = SpaltenEntf(title)
#print(title)

#b = title.loc[title[2017] > 5] 

#b.plot.bar()
#print(b)
#b.shape

In [9]:
#Einzelnes Wort anzeigen
#abstract = CreateTermDocMatrix('Abstract')
#ShowWord(abstract,'software')

In [10]:
data.columns

Index(['Document Title', 'Authors', 'Author Affiliations', 'Publication Title',
       'Date Added To Xplore', 'Publication Year', 'Volume', 'Issue',
       'Start Page', 'End Page', 'Abstract', 'ISSN', 'ISBNs', 'DOI',
       'Funding Information', 'PDF Link', 'Author Keywords', 'IEEE Terms',
       'INSPEC Controlled Terms', 'INSPEC Non-Controlled Terms', 'Mesh_Terms',
       'Article Citation Count', 'Patent Citation Count', 'Reference Count',
       'License', 'Online Date', 'Issue Date', 'Meeting Date', 'Publisher',
       'Document Identifier'],
      dtype='object')